In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [97]:
data = pd.read_csv('test.csv')

In [59]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
data.nunique()

id                                   25976
Gender                                   2
Customer Type                            2
Age                                     75
Type of Travel                           2
Class                                    3
Flight Distance                       3281
Inflight wifi service                    6
Departure/Arrival time convenient        6
Ease of Online booking                   6
Gate location                            5
Food and drink                           6
Online boarding                          6
Seat comfort                             5
Inflight entertainment                   6
On-board service                         6
Leg room service                         6
Baggage handling                         5
Checkin service                          5
Inflight service                         6
Cleanliness                              6
Departure Delay in Minutes             313
Arrival Delay in Minutes               320
satisfactio

In [98]:
data['satisfaction'] = pd.get_dummies(data['satisfaction'])
data

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,19556,Female,Loyal Customer,52,Business travel,Eco,160,5,4,...,5,5,5,5,2,5,5,50,44.0,0
1,1,90035,Female,Loyal Customer,36,Business travel,Business,2863,1,1,...,4,4,4,4,3,4,5,0,0.0,0
2,2,12360,Male,disloyal Customer,20,Business travel,Eco,192,2,0,...,2,4,1,3,2,2,2,0,0.0,1
3,3,77959,Male,Loyal Customer,44,Business travel,Business,3377,0,0,...,1,1,1,1,3,1,4,0,6.0,0
4,4,36875,Female,Loyal Customer,49,Business travel,Eco,1182,2,3,...,2,2,2,2,4,2,4,0,20.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25971,25971,78463,Male,disloyal Customer,34,Business travel,Business,526,3,3,...,4,3,2,4,4,5,4,0,0.0,1
25972,25972,71167,Male,Loyal Customer,23,Business travel,Business,646,4,4,...,4,4,5,5,5,5,4,0,0.0,0
25973,25973,37675,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,...,2,4,3,4,5,4,2,0,0.0,1
25974,25974,90086,Male,Loyal Customer,14,Business travel,Business,1127,3,3,...,4,3,2,5,4,5,4,0,0.0,0


In [99]:
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].apply(lambda x: 1 if x < 50 else 0)


In [100]:
dataA = data[data['Arrival Delay in Minutes'] == 1]['satisfaction']
dataA

0        0
1        0
2        1
3        0
4        0
        ..
25971    1
25972    0
25973    1
25974    0
25975    1
Name: satisfaction, Length: 23627, dtype: uint8

In [101]:
dataB = data[data['Arrival Delay in Minutes'] == 0]['satisfaction']

In [102]:
mean_A = dataA.mean()

In [103]:
mean_B = dataB.mean()

In [104]:
std_A = dataA.std()

In [105]:
std_B = dataB.std()

In [106]:
len_A = len(dataA)

In [107]:
len_B = len(dataB)

In [181]:
from numpy import sqrt, abs, round
from scipy.stats import norm
import scipy

## z-test-formula:

In [109]:
d = sqrt((std_A **2 / len_A) + (std_B **2 / len_B))

In [110]:
d

0.0104382011635742

In [111]:
a = mean_A - mean_B

In [112]:
z = a/d

In [113]:
z

-7.988905001023973

In [114]:
p_value = 2 * (1 - norm.cdf(abs(z)))

In [115]:
p_value

1.3322676295501878e-15

In [123]:
data.rename(columns={'Arrival Delay in Minutes': 'delay'}, inplace=True)

## Anova:

In [116]:
from statsmodels.formula.api import ols

In [124]:
res = ols('satisfaction ~ delay', data=data).fit()

In [126]:
sm.stats.anova_lm(res)

,df,sum_sq,mean_sq,F,PR(>F)
delay,1.0,14.857480,14.857480,60.464155,7.769959e-15
Residual,25974.0,6382.429207,0.245724,NaN,NaN


# what is df : degree fredoom:
## delay_df = (n-a) --> 2-1=1
## residual_df = (alldata - classtarget) --> 25976 - 2 = 25974

# ----------------------------------------------------------------------------------
# delay_sumsq: sum(all elemnet minus mean ** 2)
## residula_sumsq : sum(all elements minus by specific own class) ** 2

In [142]:
gmean = data.groupby(['delay']).mean()[['satisfaction']]

In [152]:
gmean.rename(columns={'satisfaction':'gmean'}, inplace=True)
gmean
data['mean'] = data['satisfaction'].mean()

In [156]:
data = data.iloc[:, -3:].merge(gmean, left_on='delay', right_index=True)

In [165]:
residual_sumsq = sum((data['satisfaction'] - data['gmean']) ** 2)

In [166]:
delay_sumsq = sum((data['mean'] - data['gmean']) ** 2)

In [175]:
delay_meansq = delay_sumsq / 1
delay_meansq

14.857480216784543

In [177]:
residual_mensq = residual_sumsq / 25974
residual_mensq

0.24572377021235284

In [179]:
f = delay_meansq / residual_mensq

In [185]:
p_value = (1 - scipy.stats.f.cdf(f, 1, 25974))

In [5]:
data.drop(data.iloc[:, 3:23].columns, axis=1, inplace=True)

In [6]:
data.drop('id', inplace=True, axis=1)

In [7]:
data = pd.get_dummies(data)

In [8]:
data.drop(['Gender_Male', 'Customer Type_Loyal Customer', 'satisfaction_neutral or dissatisfied'], axis=1, inplace=True)

In [16]:
data.rename(columns={'Gender_Female': 'gender', 'Customer Type_disloyal Customer':'customer', 'satisfaction_satisfied':'statisfie'}, inplace=True)

In [29]:
df2 = data.query('gender == 0  and customer == 1 ')
len(df2)

2225

In [30]:
df3 = data.query('gender == 1  and customer == 0 ')
len(df3)

10598

In [27]:
df = df2.merge(df3, how='outer')

In [28]:
df

,gender,customer,statisfie
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
12818,1,0,0
12819,1,0,0
12820,1,0,0
12821,1,0,0


In [31]:
df.nunique()

gender       2
customer     2
statisfie    2
dtype: int64

In [32]:
df_a = df.query('gender == 0  and customer == 1 ')

In [35]:
df_b = df.query('gender == 1  and customer == 0 ')

In [37]:
df_a['statisfie'].mean()

0.2606741573033708

In [38]:
df_b['statisfie'].mean()

0.48188337422155125

In [62]:
count_a = len(df_a[df_a['statisfie'] == 1])

In [63]:
count_b = len(df_b[df_b['statisfie'] == 1])

In [66]:
n_a = len(df_a)
n_b = len(df_b)

In [68]:
sm.stats.proportions_ztest([count_a, count_b], [n_a, n_b], alternative='larger')

(-19.094397966996553, 1.0)